In [7]:
import os  # 디렉터리 확인용
import time  # 시간 지연용
import re  # 정규표현식 (문자열에서 특정 패턴 찾기 위함)

# 감시할 폴더 경로
watch_folder = './monitor_directory'  #monitor_directory <- 해당 폴더 모니터링 

# 3초마다 확인
check_sec = 3  # 초

# 이메일 찾는 정규식
email_re = re.compile(r'[\w\.-]+@[\w\.-]+')

# SQL문 키워드 찾는 정규식 (단어 단위로 검사)
sql_re = re.compile(r'\b(select|insert|update|delete)\b', re.IGNORECASE)

# 시작할 때 있는 파일들 저장(초기 파일 확인용)
before_files = set(os.listdir(watch_folder))

print("폴더 모니터링 시작")
if before_files:
    print("초기 파일 목록:")
    for f in before_files:
        print(f"   - {f}")
else:
    print("초기 폴더가 비어 있습니다.")

while True: # 무한 루프(계속 폴더 감시하려고)
    time.sleep(check_sec) # 지정된 시간만큼 대기한 후 다음 작업 진행, 아까 3초로 설정해둔 것
    now_files = set(os.listdir(watch_folder))
    new_files = now_files - before_files # 이전 목록과 비교해서 새로 생긴 파일

     # 새로 추가된 파일이 있을 때
    for file in new_files:
        path = os.path.join(watch_folder, file)
        print(f"\n새 파일 탐지!: {file}")

        # 위험할 수 있는 확장자인지 확인 (.py, .js, .class 중 하나면 출력한다)
        if file.endswith(('.py', '.js', '.class')):
            print(f"주의 확장자 파일 탐지!: {file}")

        try:
            # 파일 내용을 읽어오기
            with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                data = f.read()

                # 주석 포함된 줄만 추출
                comment_lines = [line.strip() for line in data.splitlines()
                                 if '#' in line or '//' in line or '/*' in line]
                if comment_lines:
                    print("주석이 포함된 라인:")
                    for line in comment_lines:
                        print(f"   → {line}")

                # 이메일 추출
                emails = email_re.findall(data)
                if emails:
                    print("이메일 주소 탐지!:")
                    for e in emails:
                        print(f"   → {e}")

                # SQL 구문 추출
                sqls = sql_re.findall(data)
                if sqls:
                    print("SQL문 탐지!:")
                    for keyword in sqls:
                        print(f"   → {keyword}")

        except Exception as err:
            print(f"파일을 읽는 중 오류 발생: {err}")

    # 상태 업데이트
    before_files = now_files


폴더 모니터링 시작
초기 파일 목록:
   - 초기폴더 입니당

새 파일 탐지!: #안녕.js
주의 확장자 파일 탐지!: #안녕.js
주석이 포함된 라인:
   → #안녕

새 파일 탐지!: #안녕.class
주의 확장자 파일 탐지!: #안녕.class
주석이 포함된 라인:
   → #안녕

새 파일 탐지!: 그냥 일반 텍스트 파일.txt

새 파일 탐지!: 안에 내용있는 파이썬 파일.py
주의 확장자 파일 탐지!: 안에 내용있는 파이썬 파일.py
주석이 포함된 라인:
   → #주석 테스트용 주석
   → admin@naver.com #이메일 확인용
   → select * from users #SQL문 확인용
이메일 주소 탐지!:
   → admin@naver.com
SQL문 탐지!:
   → select


KeyboardInterrupt: 